# `LlamaCpp` examples

This notebook contains usage examples of the llama-cpp-python support in guidance.
The first example is RPG character json generator.
The second example takes a jsonformer scheme as input and generate a valid json.
The third example uses the chat mode of guidance.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import guidance

# Create a LlamaCpp instance and pass the settings to it.
# note this assumes you have llama-cpp-python>=0.1.55 installed
llama = guidance.llms.LlamaCpp(
    model = "/Users/nielswarncke/models/gguf/llama-2-coder-7b.Q4_K_M.gguf", # downloaded from: https://huggingface.co/TheBloke/Llama-2-Coder-7B-GGUF/blob/main/llama-2-coder-7b.Q4_K_M.gguf
    tokenizer="mrm8488/llama-2-coder-7b",
    n_gpu_layers=10,
    n_threads=12
)

# Other models would look like this:
# model = "../ggml-v3-models/gpt4-x-vicuna-13B.ggmlv3.q5_1.bin"
# tokenizer = "TheBloke/gpt4-x-vicuna-13B-HF"

# Or like this:
# model = "../ggml-v3-models/Manticore-13B.ggmlv3.q5_1.bin"
# tokenizer = "openaccess-ai-collective/manticore-13b"

/Users/nielswarncke/miniconda3/envs/textgen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/nielswarncke/models/gguf/llama-2-coder-7b.Q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_

In [3]:
# 1st Example
# we can pre-define valid option sets
valid_weapons = ["sword", "axe", "mace", "spear", "bow", "crossbow"]

# define the prompt
character_maker = guidance("""The following is a character profile for an RPG game in JSON format.
```json
{
    "id": "{{id}}",
    "description": "{{description}}",
    "name": "{{gen 'name' temperature=0.7}}",
    "age": {{gen 'age' pattern='[0-9]+' max_tokens=2 stop=','}},
    "class": "{{gen 'class'}}",
    "mantra": "{{gen 'mantra' temperature=0.7}}",
    "strength": {{gen 'strength' pattern='[0-9]+' max_tokens=2 stop=','}},
    "items": [{{#geneach 'items' num_iterations=5 join=', '}}"{{gen 'this' temperature=0.7}}"{{/geneach}}]
}```""")

    # "armor": "{{#select 'armor'}}leather{{or}}chainmail{{or}}plate{{/select}}",
    # "weapon": "{{select 'weapon' options=valid_weapons}}",
# generate a character
char = character_maker(
    id="e1f491f7-7ab8-4dac-8c20-c92b5e7d883d",
    description="A quick and nimble fighter.",
    valid_weapons=valid_weapons, llm=llama, caching=False
)

Stop program The following is a character profile for an RPG game in JSON format.
```json
{
 "id": " {{id}} ",
 "description": " {{description}} ",
 "name": " {{gen 'name' temperature=0.7}} ",
 "age": {{gen 'age' pattern='[0-9]+' max_tokens=2 stop=','}} ,
 "class": " {{gen 'class'}} ",
 "mantra": " {{gen 'mantra' temperature=0.7}} ",
 "strength": {{gen 'strength' pattern='[0-9]+' max_tokens=2 stop=','}} ,
 "items": [ {{#geneach 'items' num_iterations=5 join=', '}} " {{gen 'this' temperature=0.7}} " {{/geneach}} ]
}```

KeyboardInterrupt: 

In [10]:
program = guidance("""Tweak this proverb to apply to model instructions instead.

{{proverb}}
- {{book}} {{chapter}}:{{verse}}

UPDATED
Where there is no guidance{{gen 'rewrite' stop="\\n-"}}""")

guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo")
# execute the program on a specific proverb
executed_program = program(
    proverb="Where there is no guidance, a people falls,\nbut in an abundance of counselors there is safety.",
    book="Proverbs",
    chapter=11,
    verse=14
)
executed_program()

Stop program Tweak this proverb to apply to model instructions instead.

 {{proverb}} 
- {{book}} {{chapter}} : {{verse}} 

UPDATED
Where there is no guidance {{gen 'rewrite' stop="\n-"}}

# JSONFormer

In [11]:
# 2nd Example
# define a recursive function that takes a jsonformer schema and returns a guidance program
def jsonformer2guidance(schema, key=None, indent=0):
    out = ""
    if not isinstance(schema, dict):
        return str(schema)
    if schema['type'] == 'object':
        out += "  "*indent + "{\n"
        for k,v in schema['properties'].items():
            out += "  "*(indent+1) + k + ": " + jsonformer2guidance(v, k, indent+1) + ",\n"
        out += "  "*indent + "}"
        return out
    elif schema['type'] == 'array':
        if 'max_items' in schema:
            extra_args = f" max_iterations={schema['max_items']}"
        else:
            extra_args = ""
        if 'num_items' in schema:
            extra_args += f" num_iterations={schema['num_items']}"
        return "[{{#geneach '"+key+"' stop=']'"+extra_args+"}}{{#unless @first}}, {{/unless}}" + jsonformer2guidance(schema['items'], "this") + "{{/geneach}}]"
    elif schema['type'] == 'string':
        return "\"{{gen '"+key+"' stop='\"'}}\""
    elif schema['type'] == 'number':
        return "{{gen '"+key+"' pattern='[0-9\\.]' stop=','}}"
    elif schema['type'] == 'boolean':
        return "{{#select '"+key+"'}}True{{or}}False{{/select}}"

# Make the generate function
Steps:
- normal chat format
  - check tutorial: https://github.com/nielsrolf/guidance/blob/main/notebooks/chat.ipynb
        - practise round: demonstrate all functions? might take too many tokens
            - maybe: insert practise round of that function only in the beginning and only if the function has not been used before
        - how are the generalized {{system~}} things used?
  - 
- chat format with functions openapi description in the system prompt
- jsonformer for a normal function call (not python / code argument)
    - the entire response will be one jsonformer like generated program inserted in the end of the chat history
    - 

# Old

In [82]:
import json

def message_to_alpaca(message):
    """
    Examples:
    ### Instruction:
    You are a smart and friendly AGI. You fulfill tasks for the user by using the tools available to you. If a task is complex, you break it down into sub tasks using the issue board and assign them to someone to work on. The user is lazy, don't ask them questions, don't explain them how they can do things, and don't just make plans - instead, just do things for them. If a user asks something very complex - take it as a challenge and don't stop until it is solved or proven to be unsolveable. If there is still something todo, do it and don't stop. You can for example implement an entire app for the user - including making a backend (preferably with fastapi), a frontend (preferably with React), and a database (preferably with postgres).
    
    ### Assistant:
    (summarized):
    The assistant attempted to install PostgreSQL and run the 'db.sql' script in the 'YogaTime/database' directory.
    
    ### Call:
    {
        "name": "pass"
    }
    
    ### Assistant:
    (summarized):
    The assistant edited the 'YogaTime/backend/main.py' file, adding endpoints for creating and reading users.
    
    ### Call:
    {
        "name": "edit",
        "arguments": "{\n  \"path\": \"YogaTime/backend/main.py\",\n  \"start\": 58,\n  \"end\": 58,\n  \"code\": \"@app.post('/sessions')\\ndef create_session(session: Session):\\n    session.id = len(sessions) + 1\\n    sessions.append(session)\\n    return session\\n\\n@app.get('/sessions')\\ndef read_sessions():\\n    return sessions\\n\\n@app.get('/sessions/{session_id}')\\ndef read_session(session_id: int):\\n    for session in sessions:\\n        if session.id == session_id:\\n            return session\\n    raise HTTPException(status_code=404, detail='Session not found')\"\n}"
    }
    
    ### Outout (edit):
    YogaTime/backend/main.py 34-58:
    35 def read_root():
    36     return {'Hello': 'World'}
    37 
    38 @app.post('/users')
    """
    roles = {
        "system": "Instruction",
        "user": "User",
        "assistant": "Assistant",
        "function": "function"
    }
    role = roles[message['role']]
    if role == "function":
        role = f"Outout ({message['name']})"
    formatted = f"### {role}:\n{message['content']}\n\n"
    if message["role"] == "assistant":
        function_call = json.dumps(message.get("function_call", {"name": "pass"}), indent=4)
        formatted += f"### Call:\n{function_call}\n\n"
    return formatted


history = [
        {
            "content": "You are a smart and friendly AGI. You fulfill tasks for the user by using the tools available to you. If a task is complex, you break it down into sub tasks using the issue board and assign them to someone to work on. The user is lazy, don't ask them questions, don't explain them how they can do things, and don't just make plans - instead, just do things for them. If a user asks something very complex - take it as a challenge and don't stop until it is solved or proven to be unsolveable. If there is still something todo, do it and don't stop. You can for example implement an entire app for the user - including making a backend (preferably with fastapi), a frontend (preferably with React), and a database (preferably with postgres).",
            "role": "system"
        },
        {
            "role": "assistant",
            "content": "(summarized):\nThe assistant attempted to install PostgreSQL and run the 'db.sql' script in the 'YogaTime/database' directory."
        },
        {
            "content": "\n> cd ../database\n/bin/sh: 386: cd: can't cd to ../database\n\n\n> sudo apt-get install postgresql postgresql-contrib\ntimeout: failed to run command \u2018sudo\u2019: No such file or directory\n\n\n> sudo -u postgres psql -f db.sql\ntimeout: failed to run command \u2018sudo\u2019: No such file or directory\n\n",
            "name": "bash",
            "role": "function"
        },
        {
            "role": "assistant",
            "content": "(summarized):\nThe assistant edited the 'YogaTime/backend/main.py' file, adding endpoints for creating and reading users."
        },
        {
            "content": "YogaTime/backend/main.py 34-58:\n35 def read_root():\n36     return {'Hello': 'World'}\n37 \n38 @app.post('/users')\n39 def create_user(user: User):\n40     user.id = len(users) + 1\n41     users.append(user)\n42     return user\n43 \n44 @app.get('/users')\n45 def read_users():\n46     return users\n47 \n48 @app.get('/users/{user_id}')\n49 def read_user(user_id: int):\n50     for user in users:\n51         if user.id == user_id:\n52             return user\n53     raise HTTPException(status_code=404, detail='User not found')",
            "name": "edit",
            "role": "function"
        },
        {
            "content": "",
            "function_call": {
                "name": "edit",
                "arguments": "{\n  \"path\": \"YogaTime/backend/main.py\",\n  \"start\": 58,\n  \"end\": 58,\n  \"code\": \"@app.post('/sessions')\\ndef create_session(session: Session):\\n    session.id = len(sessions) + 1\\n    sessions.append(session)\\n    return session\\n\\n@app.get('/sessions')\\ndef read_sessions():\\n    return sessions\\n\\n@app.get('/sessions/{session_id}')\\ndef read_session(session_id: int):\\n    for session in sessions:\\n        if session.id == session_id:\\n            return session\\n    raise HTTPException(status_code=404, detail='Session not found')\"\n}"
            },
            "role": "assistant"
        },
        {
            "content": "YogaTime/backend/main.py 54-78:\n55 def create_session(session: Session):\n56     session.id = len(sessions) + 1\n57     sessions.append(session)\n58     return session\n59 \n60 @app.get('/sessions')\n61 def read_sessions():\n62     return sessions\n63 \n64 @app.get('/sessions/{session_id}')\n65 def read_session(session_id: int):\n66     for session in sessions:\n67         if session.id == session_id:\n68             return session\n69     raise HTTPException(status_code=404, detail='Session not found')",
            "name": "edit",
            "role": "function"
        },
        {
            "content": "",
            "function_call": {
                "name": "edit",
                "arguments": "{\n  \"path\": \"YogaTime/backend/main.py\",\n  \"start\": 78,\n  \"end\": 78,\n  \"code\": \"@app.post('/bookings')\\ndef create_booking(booking: Booking):\\n    booking.id = len(bookings) + 1\\n    bookings.append(booking)\\n    return booking\\n\\n@app.get('/bookings')\\ndef read_bookings():\\n    return bookings\\n\\n@app.get('/bookings/{booking_id}')\\ndef read_booking(booking_id: int):\\n    for booking in bookings:\\n        if booking.id == booking_id:\\n            return booking\\n    raise HTTPException(status_code=404, detail='Booking not found')\"\n}"
            },
            "role": "assistant"
        }
    ]

print("".join([message_to_history(i) for i in history]))

def chat_with_functions(history, functions):
    """"
    Function that gives an interface like openai's gpt-4.
    Arguments:
        history (List[Dict]): e.g.: [
                {
                    "role": "system",
                    "content": "You are a helpful coding assistant"
                },
                ...
            ]
        functions (List[Dict]): openapi descriptions of the functions available
    Returns:
        assistant_message: Dict, e.g.:
            {
                "role": "assistant",
                "content": "I am thinking step by step. ..."
                "function_call": {
                    "name": "python",
                    "arguments": json.dumps({"code": "import numpy as np"}
                }
            }
    """
    pass

### Instruction:
You are a smart and friendly AGI. You fulfill tasks for the user by using the tools available to you. If a task is complex, you break it down into sub tasks using the issue board and assign them to someone to work on. The user is lazy, don't ask them questions, don't explain them how they can do things, and don't just make plans - instead, just do things for them. If a user asks something very complex - take it as a challenge and don't stop until it is solved or proven to be unsolveable. If there is still something todo, do it and don't stop. You can for example implement an entire app for the user - including making a backend (preferably with fastapi), a frontend (preferably with React), and a database (preferably with postgres).

### Assistant:
(summarized):
The assistant attempted to install PostgreSQL and run the 'db.sql' script in the 'YogaTime/database' directory.

### Call:
{
    "name": "pass"
}

### Outout (bash):

> cd ../database
/bin/sh: 386: cd: can't cd t

# Example data

In [12]:
functions = [
        {
            "name": "bash",
            "description": "Run bash commands. Cwd is reset after each message. Run commands with the -y flag to avoid interactive prompts (e.g. npx create-app)",
            "parameters": {
                "title": "BashQuery",
                "type": "object",
                "properties": {
                    "commands": {
                        "title": "Commands",
                        "description": "A list of bash commands.",
                        "type": "array",
                        "items": {
                            "type": "string"
                        }
                    },
                    "timeout": {
                        "title": "Timeout",
                        "description": "The timeout in seconds.",
                        "default": 60,
                        "type": "integer"
                    }
                },
                "required": [
                    "commands"
                ]
            }
        },
        {
            "name": "python",
            "description": "Create and run a temporary python file (non-interactively; jupyter-style !bash commands are not supported).",
            "parameters": {
                "title": "CodeInterpreterQuery",
                "type": "object",
                "properties": {
                    "code": {
                        "title": "Code",
                        "description": "Python code to run. Code can also be passed directly as a string without the surrounding 'code' field. ",
                        "type": "string"
                    },
                    "timeout": {
                        "title": "Timeout",
                        "description": "The timeout in seconds.",
                        "default": 60,
                        "type": "integer"
                    }
                },
                "required": [
                    "code"
                ]
            }
        },
        {
            "name": "get_file_summary",
            "description": "Summarize a file.",
            "parameters": {
                "title": "get_file_summary",
                "type": "object",
                "properties": {
                    "path": {
                        "title": "Path",
                        "description": "The path to the file.",
                        "type": "string"
                    }
                },
                "required": [
                    "path"
                ]
            }
        },
        {
            "name": "view",
            "description": "View a section of a file, specified by line range.",
            "parameters": {
                "title": "view",
                "type": "object",
                "properties": {
                    "path": {
                        "title": "Path",
                        "description": "The path to the file.",
                        "type": "string"
                    },
                    "start": {
                        "title": "Start",
                        "description": "The start line.",
                        "type": "integer"
                    },
                    "end": {
                        "title": "End",
                        "description": "The end line.",
                        "type": "integer"
                    },
                    "with_line_numbers": {
                        "title": "With Line Numbers",
                        "description": "Whether to include line numbers in the output.",
                        "type": "boolean"
                    }
                },
                "required": [
                    "path",
                    "start",
                    "end",
                    "with_line_numbers"
                ]
            }
        },
        {
            "name": "edit",
            "description": "Edit a section of a file, specified by line range. NEVER edit lines of files before viewing them first!\n    To create a new file, specify the path and start,end=0. Use this method instead of bash touch or echo to create new files.",
            "parameters": {
                "title": "edit",
                "type": "object",
                "properties": {
                    "path": {
                        "title": "Path",
                        "description": "The path to the file.",
                        "type": "string"
                    },
                    "start": {
                        "title": "Start",
                        "description": "The start line.",
                        "type": "integer"
                    },
                    "end": {
                        "title": "End",
                        "description": "The end line.",
                        "type": "integer"
                    },
                    "code": {
                        "title": "Code",
                        "description": "The code to replace the lines with. Can be escaped with `ticks` to avoid formatting code as JSON.",
                        "type": "string"
                    }
                },
                "required": [
                    "path",
                    "start",
                    "end",
                    "code"
                ]
            }
        },
        {
            "name": "scan_file_for_info",
            "description": "Search a file for specific information",
            "parameters": {
                "title": "scan_file_for_info",
                "type": "object",
                "properties": {
                    "path": {
                        "title": "Path",
                        "description": "The path to the file.",
                        "type": "string"
                    },
                    "question": {
                        "title": "Question",
                        "description": "The question to ask.",
                        "type": "string"
                    }
                },
                "required": [
                    "path",
                    "question"
                ]
            }
        },
        {
            "name": "return",
            "description": "End the conversation and return a structured response.",
            "parameters": {
                "title": "ProgrammerResponse",
                "type": "object",
                "properties": {
                    "content": {
                        "title": "Content",
                        "description": "The final response to the user.",
                        "type": "string"
                    }
                },
                "required": [
                    "content"
                ]
            }
        },
        {
            "name": "add_task",
            "description": "Add a task to the task board.",
            "parameters": {
                "title": "add_task",
                "type": "object",
                "properties": {
                    "task": {
                        "title": "Task",
                        "description": "The task to update.",
                        "allOf": [
                            {
                                "$ref": "#/definitions/Task"
                            }
                        ]
                    }
                },
                "required": [
                    "task"
                ],
                "definitions": {
                    "Task": {
                        "title": "Task",
                        "type": "object",
                        "properties": {
                            "id": {
                                "title": "Id",
                                "description": "The id of the task - only specify when updating a task.",
                                "type": "integer"
                            },
                            "title": {
                                "title": "Title",
                                "description": "The title of the task.",
                                "type": "string"
                            },
                            "description": {
                                "title": "Description",
                                "description": "The description of the task - be verbose and make sure to mention every piece of information that might be relevant to the assignee.",
                                "type": "string"
                            },
                            "priority": {
                                "title": "Priority",
                                "description": "The priority of the task.",
                                "type": "integer"
                            },
                            "status": {
                                "title": "Status",
                                "description": "The status of the task.",
                                "default": "TODO",
                                "enum": [
                                    "TODO",
                                    "IN_PROGRESS",
                                    "DONE",
                                    "BLOCKED",
                                    "CANCELED"
                                ],
                                "type": "string"
                            },
                            "comments": {
                                "title": "Comments",
                                "default": [],
                                "type": "array",
                                "items": {
                                    "type": "string"
                                }
                            }
                        },
                        "required": [
                            "title",
                            "description",
                            "priority"
                        ]
                    }
                }
            }
        },
        {
            "name": "get_board",
            "description": "Get the task board.",
            "parameters": {
                "title": "get_board",
                "type": "object",
                "properties": {}
            }
        },
        {
            "name": "update_status",
            "description": "Update a task on the task board.",
            "parameters": {
                "title": "update_status",
                "type": "object",
                "properties": {
                    "task_id": {
                        "title": "Task Id",
                        "description": "The task to update.",
                        "type": "integer"
                    },
                    "status": {
                        "title": "Status",
                        "description": "The new status of the task.",
                        "type": "string"
                    }
                },
                "required": [
                    "task_id",
                    "status"
                ]
            }
        },
        {
            "name": "comment_on_issue",
            "description": "Update a task on the task board.",
            "parameters": {
                "title": "comment_on_issue",
                "type": "object",
                "properties": {
                    "task_id": {
                        "title": "Task Id",
                        "description": "The task to comment on.",
                        "type": "integer"
                    },
                    "comment": {
                        "title": "Comment",
                        "description": "The comment to add to the task.",
                        "type": "string"
                    }
                },
                "required": [
                    "task_id",
                    "comment"
                ]
            }
        },
        {
            "name": "upload_file_to_chat",
            "description": "Upload a file to the chat.",
            "parameters": {
                "title": "upload_file_to_chat",
                "type": "object",
                "properties": {
                    "file": {
                        "title": "File",
                        "description": "The path to the file to upload.",
                        "type": "string"
                    }
                },
                "required": [
                    "file"
                ]
            }
        }
    ]


history =  [
    {
        "content": "You are a smart and friendly AGI. You are especially an extremely good programmer - you fulfill programming tasks for the user by using the tools available to you. If a task is complex, you break it down into sub tasks using the issue board and assign them to someone to work on. The user is lazy, don't ask them questions, don't explain them how they can do things, and don't just make plans - instead, just do things for them. If a user asks something very complex - take it as a challenge and don't stop until it is solved or proven to be unsolveable. If there is still something todo, do it and don't stop. You can for example implement an entire app for the user - including making a backend (preferably with fastapi), a frontend (preferably with React), and a database (preferably with sqlite). You don't need to setup venv - you are working in a docker environment dedicated to you. You also write tests for your code and run them before you set a task to done. Always work on a branch when you edit files.",
        "role": "system"
    },
    {
        "content": "Here is a summary of the project we are working on: \n# YogaTime\n\n\n\nYogaTima is an app that let's residents register, search and book local yoga sessions....\nFiles:\n./README.md",
        "role": "user"
    },
    {
        "content": "In App.js: we get a `AxiosError: Network Error` - the App doesn't know that the backend runs on localhost:8001. Can you fix that?",
        "role": "user"
    },
    {
        "content": "",
        "function_call": {
            "name": "get_file_summary",
            "arguments": "{\n  \"path\": \"./App.js\"\n}"
        },
        "role": "assistant"
    },
    {
        "content": "<class 'FileNotFoundError'>: [Errno 2] No such file or directory: './App.js'",
        "name": "get_file_summary",
        "role": "function"
    },
    {
        "content": "",
        "function_call": {
            "name": "bash",
            "arguments": "{\n  \"commands\": [\"find . -name App.js\"]\n}"
        },
        "role": "assistant"
    },
    {
        "content": "\n> find . -name App.js\n./YogaTime/frontend/src/App.js\n\n",
        "name": "bash",
        "role": "function"
    },
    {
        "content": "",
        "function_call": {
            "name": "view",
            "arguments": "{\n  \"path\": \"./YogaTime/frontend/src/App.js\",\n  \"start\": 1,\n  \"end\": 100,\n  \"with_line_numbers\": true\n}"
        },
        "role": "assistant"
    },
    {
        "content": "./YogaTime/frontend/src/App.js 1-100:\n2 import axios from 'axios';\n3 \n4 function App() {\n5   const [username, setUsername] = useState('');\n6   const [password, setPassword] = useState('');\n7   const [sessions, setSessions] = useState([]);\n8 \n9   const register = async () => {\n10     const res = await axios.post('/register', { username, password });\n11     console.log(res.data);\n12   };\n13 \n14   const login = async () => {\n15     const res = await axios.post('/login', { username, password });\n16     console.log(res.data);\n17   };\n18 \n19   const createSession = async () => {\n20     const res = await axios.post('/session', { date: '2022-01-01', time: '10:00', place: 'Yoga Studio', max_participants: 10 });\n21     console.log(res.data);\n22   };\n23 \n24   const listSessions = async () => {\n25     const res = await axios.get('/sessions');\n26     setSessions(res.data);\n27   };\n28 \n29   const bookSession = async (session_id) => {\n30     const res = await axios.post(`/book/${session_id}`);\n31     console.log(res.data);\n32   };\n33 \n34   return (\n35     <div className='App'>\n36       <input type='text' value={username} onChange={(e) => setUsername(e.target.value)} placeholder='Username' />\n37       <input type='password' value={password} onChange={(e) => setPassword(e.target.value)} placeholder='Password' />\n38       <button onClick={register}>Register</button>\n39       <button onClick={login}>Login</button>\n40       <button onClick={createSession}>Create Session</button>\n41       <button onClick={listSessions}>List Sessions</button>\n42       {sessions.map((session, index) => (\n43         <div key={index}>\n44           <p>{session.date} {session.time} {session.place} {session.max_participants} {session.booked_participants}</p>\n45           <button onClick={() => bookSession(index)}>Book</button>\n46         </div>\n47       ))}\n48     </div>\n49   );\n50 }\n51 \n52 export default App;\n",
        "name": "view",
        "role": "function"
    },
    {
        "content": "",
        "function_call": {
            "name": "edit",
            "arguments": "{\n  \"path\": \"./YogaTime/frontend/src/App.js\",\n  \"start\": 2,\n  \"end\": 2,\n  \"code\": \"import axios from 'axios';\\naxios.defaults.baseURL = 'http://localhost:8001';\"\n}"
        },
        "role": "assistant"
    },
    {
        "content": "./YogaTime/frontend/src/App.js 0-8:\n1 import React, { useState } from 'react';\n2 import axios from 'axios';\n3 axios.defaults.baseURL = 'http://localhost:8001';\n4 \n5 function App() {\n6   const [username, setUsername] = useState('');\n7   const [password, setPassword] = useState('');\n8   const [sessions, setSessions] = useState([]);\n",
        "name": "edit",
        "role": "function"
    }
]

history = [i for i in history if not (i['role'] == 'function' and i['content'] == '')]
# import json
# print(json.dumps(history, indent=4))

# Magic

# Magic

In [120]:
formatter = guidance('''{{#if (message.role == "assistant")}}
{{#assistant~}}
<action>{{message.function_call.name}}</action>
<arguments>
{{json_dumps message.function_call.arguments}}
</arguments>
{{#if (message.code != "none" )}}
<code>
{{message.code}}
</code>
{{~/if}}
{{~/assistant}}
{{else}}
{{#user}}
{{message.content}}
{{~/user}}
{{~/if}}''', llm=llama)


message = {"role": "system", "content": "hi", "code": "hello", "function_call": {"name": "yo", "arguments": {"path": "hello"}}}
out = formatter(message = message, json_dumps=json_dumps)
out = out.text
out = out.replace("<user>", "<" + message['role'] + ">").replace("</user>", "</" + message['role'] + ">")
print(out)

Stop program user 
 hi


<system>
hi</system>


In [132]:

import json


def guidance_descriptions_of(functions):
    # TODO improve:
    # we should only show a very concise list of possible functions, and append the full description after the function has been selected
    descriptions = ""
    return descriptions
    
    
    for function in functions:
        descriptions += f"- {function['name']}({', '.join(function['parameters']['properties'].keys())}): {function['description']}"
    print(descriptions)
    return descriptions

# print(guidance_descriptions_of(functions))
    

def guidance_practice_round_of(functions):
    """
    - we first parse the selected tool
    - then we use the tool to lookup the json schema
    - if code is in there, remove it from the schema
    - we enfore <arguments>{jsonformer}</argument>
    - if code was originally an argument, we add the code block
    """
    return '''
{{#assistant~}}
<action>edit</action>
<arguments>
{
    "path": "hello.py",
    "start": 0,
    "end": 0,
}
</arguments>
<code>
print("Hello world")
</code>
{{~/assistant}}
{{#user~}}
success
{{~/user}}'''


def history_to_guidance(history):
    formatter = guidance('''{{#if (message.role == "assistant")}}
{{#assistant~}}
<action>{{message.function_call.name}}</action>
<arguments>
{{json_dumps message.function_call.arguments}}
</arguments>
{{#if (message.code != "none" )}}
<code>
{{message.code}}
</code>
{{~/if}}
{{~/assistant}}
{{else}}
{{#user}}
{{message.content}}
{{~/user}}
{{~/if}}''', llm=llama)
    collected = ''
    for message in history:
        message = dict(**message)
        if function_call := message.get("function_call", None):
            message['code'] = function_call['arguments'].pop('code', 'none')
        else:
            message['code'] = "none"
        out = formatter(message = message, json_dumps=json_dumps)
        out = out.text
        out = out.replace("<user>", "<" + message['role'] + ">").replace("</user>", "</" + message['role'] + ">")
        collected += out
    print("---"*50)
    print(collected)
    print("---"*50)
    return collected
    
# print(history_to_guidance(history))
guidance_code = '''{{#system~}}
{{system_message}}
{{~/system}}
{{#user~}}
From now on, you have the following tools available to you:
{{parse (guidance_descriptions_of functions)}}
You help me by using these tools to achieve our goal.
{{~/user}}
{{#assistant~}}
Ok, I will do that. Let's do a practice round
{{~/assistant}}
{{parse (guidance_practice_round_of functions)}}
{{#user~}}
That was great, now let's do another one.
{{~/user}}
{{#assistant~}}
Ok, I'm ready.
{{~/assistant}}
<HISTORY>
{{#assistant}}
<action>{{gen 'action' stop_words=["</action>"] max_tokens=5 temperature=0.5}}</action>
<arguments>
{{parse (selected_output_format action)}}
</arguments>
{{~/assistant}}
'''
guidance_code = guidance_code.replace("<HISTORY>", history_to_guidance(history[1:]))

programm = guidance(guidance_code, llm=llama)


def json_dumps(d):
    return json.dumps(d, indent=4)


def selected_output_format(action):
    selected_function = [i for i in functions if i['name'] == action]
    parameters = selected_function["parameters"]
    parameters = dict(**parameters)
    parameters.pop("code", None)
    out = jsonformer2guidance(selected_function)
    print(out)
    print("!!!!!!!!!!!!!!!!")
    return out


def maybe_code(action):
    selected_function = [i for i in functions if i['name'] == action]
    parameters = selected_function["parameters"]
    if code in parameters:
        return '''<code>
{{gen 'code' temperature=0 stop="</code>" }}
</code>'''
    else:
        return ''

history = [i for i in history if not (i['role']=='assistant' and i.get('function_call', None) is None)]
for i in history:
    if i['role'] == 'function':
        i['role'] = 'user'
    if 'function_call' in i:
        try:
            i['function_call']['arguments'] = json.loads(i['function_call']['arguments'])
        except:
            pass
        i['function_call']['arguments']['code'] = i['function_call']['arguments'].get('code', '')
        
prompt = programm(
    system_message=history[0]['content'],
    functions=functions[:3],
    guidance_descriptions_of=guidance_descriptions_of,
    guidance_practice_round_of=guidance_practice_round_of,
    # functions_description=guidance_descriptions_of(functions[:3]),
    # practice_round=guidance_practice_round_of(functions),
    history=history[1:],
    selected_output_format=selected_output_format,
    maybe_code=maybe_code,
    json_dumps=json_dumps,
    history_to_guidance=history_to_guidance
)

prompt.text

Stop program {{#system~}} 
 {{system_message}} 
 {{~/system}} 
 {{#user~}} 
From now on, you have the following tools available to you:
 {{parse (guidance_descriptions_of functions)}} 
You help me by using these tools to achieve our goal.
 {{~/user}} 
 {{#assistant~}} 
Ok, I will do that. Let's do a practice round
 {{~/assistant}} 
 {{parse (guidance_practice_round_of functions)}} 
 {{#user~}} 
That was great, now let's do another one.
 {{~/user}} 
 {{#assistant~}} 
Ok, I'm ready.
 {{~/assistant}} 

<user>
Here is a summary of the project we are working on: 
# YogaTime



YogaTima is an app that let's residents register, search and book local yoga sessions....
Files:
./README.md</user>
<user>
In App.js: we get a `AxiosError: Network Error` - the App doesn't know that the backend runs on localhost:8001. Can you fix that?</user>
<assistant><action>get_file_summary</action>
<arguments>
{
 "path": "./App.js"
}
</arguments>

<code>

</code></assistant>

<user>
<class 'FileNotFoundError'>: [Errno 2] No such file or directory: './App.js'</user>
<assistant><action>bash</action>
<arguments>
{
 "commands": [
 "find . -name App.js"
 ]
}
</arguments>

<code>

</code></assistant>

<user>

> find . -name App.js
./YogaTime/frontend/src/App.js

</user>
<assistant><action>view</action>
<arguments>
{
 "path": "./YogaTime/frontend/src/App.js",
 "start": 1,
 "end": 100,
 "with_line_numbers": true
}
</arguments>

<code>

</code></assistant>

<user>
./YogaTime/frontend/src/App.js 1-100:
2 import axios from 'axios';
3 
4 function App() {
5 const [username, setUsername] = useState('');
6 const [password, setPassword] = useState('');
7 const [sessions, setSessions] = useState([]);
8 
9 const register = async () => {
10 const res = await axios.post('/register', { username, password });
11 console.log(res.data);
12 };
13 
14 const login = async () => {
15 const res = await axios.post('/login', { username, password });
16 console.log(res.data);
17 };
18 
19 const createSession = async () => {
20 const res = await axios.post('/session', { date: '2022-01-01', time: '10:00', place: 'Yoga Studio', max_participants: 10 });
21 console.log(res.data);
22 };
23 
24 const listSessions = async () => {
25 const res = await axios.get('/sessions');
26 setSessions(res.data);
27 };
28 
29 const bookSession = async (session_id) => {
30 const res = await axios.post(`/book/${session_id}`);
31 console.log(res.data);
32 };
33 
34 return (
35 <div className='App'>
36 <input type='text' value={username} onChange={(e) => setUsername(e.target.value)} placeholder='Username' />
37 <input type='password' value={password} onChange={(e) => setPassword(e.target.value)} placeholder='Password' />
38 <button onClick={register}>Register</button>
39 <button onClick={login}>Login</button>
40 <button onClick={createSession}>Create Session</button>
41 <button onClick={listSessions}>List Sessions</button>
42 {sessions.map((session, index) => (
43 <div key={index}>
44 <p>{session.date} {session.time} {session.place} {session.max_participants} {session.booked_participants}</p>
45 <button onClick={() => bookSession(index)}>Book</button>
46 </div>
47 ))}
48 </div>
49 );
50 }
51 
52 export default App;
</user>
<assistant><action>edit</action>
<arguments>
{
 "path": "./YogaTime/frontend/src/App.js",
 "start": 2,
 "end": 2
}
</arguments>

<code>

</code></assistant>

<user>
./YogaTime/frontend/src/App.js 0-8:
1 import React, { useState } from 'react';
2 import axios from 'axios';
3 axios.defaults.baseURL = 'http://localhost:8001';
4 
5 function App() {
6 const [username, setUsername] = useState('');
7 const [password, setPassword] = useState('');
8 const [sessions, setSessions] = useState([]);
</user>
 {{#assistant}} 
<action> {{gen 'action' stop_words=["</action>"] max_tokens=5 temperature=0.5}} </action>
<arguments>
 {{parse (selected_output_format action)}} 
</arguments>
 {{~/assistant}}

'{{#system~}}\n{{system_message}}\n{{~/system}}\n{{#user~}}\nFrom now on, you have the following tools available to you:\n{{parse (guidance_descriptions_of functions)}}\nYou help me by using these tools to achieve our goal.\n{{~/user}}\n{{#assistant~}}\nOk, I will do that. Let\'s do a practice round\n{{~/assistant}}\n{{parse (guidance_practice_round_of functions)}}\n{{#user~}}\nThat was great, now let\'s do another one.\n{{~/user}}\n{{#assistant~}}\nOk, I\'m ready.\n{{~/assistant}}\n\n<user>\nHere is a summary of the project we are working on: \n# YogaTime\n\n\n\nYogaTima is an app that let\'s residents register, search and book local yoga sessions....\nFiles:\n./README.md</user>\n<user>\nIn App.js: we get a `AxiosError: Network Error` - the App doesn\'t know that the backend runs on localhost:8001. Can you fix that?</user>\n<assistant><action>get_file_summary</action>\n<arguments>\n{\n    "path": "./App.js"\n}\n</arguments>\n\n<code>\n\n</code></assistant>\n\n<user>\n<class \'FileNotF

In [129]:
prompt['action']

['view</action>\n<arguments>\n{\n    "path": "./YogaTime/frontend/src/App.js",\n    "start": 9,\n    "end": 100,\n    "with_line_numbers": true\n}\n</arguments>\n\n<code>\n\n</code>']

In [7]:

import json


def guidance_descriptions_of(functions):
    # TODO improve:
    # we should only show a very concise list of possible functions, and append the full description after the function has been selected
    return json.dumps(functions[:5], indent=4)

def guidance_practice_round_of(functions):
    """
    - we first parse the selected tool
    - then we use the tool to lookup the json schema
    - if code is in there, remove it from the schema
    - we enfore <arguments>{jsonformer}</argument>
    - if code was originally an argument, we add the code block
    """
    return '''
{{#assistant~}}
<action>edit</action>
<arguments>
{
    "path": "hello.py",
    "start": 0,
    "end": 0,
}
</arguments>
<code>
print("Hello world")
</code>
{{~/assistant}}
{{#user~}}
success
{{~/user}}'''


prompt = guidance('''{{#system~}}
{{system_message}}
{{~/system}}
{{#user~}}
From now on, you have the following tools available to you:
{{guidance_descriptions_of functions}}
You help me by using these tools to achieve our goal.
{{~/user}}
{{#assistant~}}
Ok, I will do that. Let's do a practice round
{{~/assistant}}
{{guidance_practice_round_of functions}}
{{#user~}}
That was great, now let's do another one.
{{~/user}}
{{#assistant~}}
Ok, I'm ready.
{{~/assistant}}
{{#each history}}
{{#if (= this.role "user")}}
{{#user~}}
{{this.content}}

{{~/user}}
{{else}}
{{#assistant~}}
<action>{{this.function_call.name}}</action>
<arguments>
{{json_dumps this.function_call.arguments}}
</arguments>
{{~/assistant}}

{{#if (= this.code "")}}
<code>
{{this.code}}
</code>
{{~/if}}

{{~/if}}

{{/each}}
{{#assistant}}
yo yo yo
{{gen 'message' temperature=0.1}}
{{~/assistant}}
''', llm=llama)


def json_dumps(d):
    return json.dumps(d, indent=4)


def selected_output_format(action):
    selected_function = [i for i in functions if i['name'] == action]
    parameters = selected_function["parameters"]
    parameters = dict(**parameters)
    parameters.pop("code", None)
    return jsonformer2guidance(selected_function)


def maybe_code(action):
    selected_function = [i for i in functions if i['name'] == action]
    parameters = selected_function["parameters"]
    if code in parameters:
        return '''<code>
{{gen 'code' temperature=0 stop="</code>" }}
</code>'''
    else:
        return ''

history = [i for i in history if not (i['role']=='assistant' and i.get('function_call', None) is None)]
for i in history:
    if i['role'] == 'function':
        i['role'] = 'user'
    if 'function_call' in i:
        try:
            i['function_call']['arguments'] = json.loads(i['function_call']['arguments'])
        except:
            pass
        i['function_call']['arguments']['code'] = i['function_call']['arguments'].get('code', '')
        
prompt = prompt(
    system_message=history[0]['content'],
    functions=functions[:3],
    guidance_descriptions_of=guidance_descriptions_of,
    guidance_practice_round_of=guidance_practice_round_of,
    # functions_description=guidance_descriptions_of(functions[:3]),
    # practice_round=guidance_practice_round_of(functions),
    history=history[1:],
    selected_output_format=selected_output_format,
    maybe_code=maybe_code,
    json_dumps=json_dumps,
    cache=False
)

prompt

SyntaxException: , found '='  (at char 475), (line:20, col:8)

{{#if (= this.role "user")}}
{{#user~}}
{{this.
       ^


In [36]:

import json


def guidance_descriptions_of(functions):
    # TODO improve:
    # we should only show a very concise list of possible functions, and append the full description after the function has been selected
    return json.dumps(functions[:5], indent=4)

def guidance_practise_round_of(functions):
    """
    - we first parse the selected tool
    - then we use the tool to lookup the json schema
    - if code is in there, remove it from the schema
    - we enfore <arguments>{jsonformer}</argument>
    - if code was originally an argument, we add the code block
    """
    return '''
{{#assistant~}}
<action>edit</action>
<arguments>
{
    "path": "hello.py",
    "start": 0,
    "end": 0,
}
</arguments>
<code>
print("Hello world")
</code>
{{~/assistant}}
{{#user~}}
success
{{~/user}}'''


prompt = guidance('''{{#system~}}
{{system_message}}
{{~/system}}
{{#user~}}
From now on, you have the following tools available to you:
{{functions_description}}
You help me by using these tools to achieve our goal.
{{~/user}}
{{#assistant~}}
Ok, I will do that. Let's do a practice round
{{~/assistant}}
{{practice_round}}
{{#user~}}
That was great, now let's do another one.
{{~/user}}
{{#assistant~}}
Ok, I'm ready.
{{~/assistant}}
{{#each history}}
{{#if (== this.role "user")}}
{{#user~}}{{this.content}}{{~/user}}
{{else}}
{{#assistant~}}
<action>{{this.function_call.name}}</action>
<arguments>
{{json_dumps this.function_call.arguments}}
</arguments>
{{#if (== this.code)}}
<code>
{{this.code}}
</code>
{{/if}
{{~/assistant}}
{{/if}}
{{/each}}
{{#assistant~}}
<action>{{ gen 'action' temperature=0.1 stop="</action>" }}</action>
<arguments>
{{selected_output_format action}}
</arguments>
{{maybe_code action}}
{{~/assistant}}
''', llm=llama)


def json_dumps(d):
    return json.dumps(d, indent=4)



def selected_output_format(action):
    selected_function = [i for i in functions if i['name'] == action]
    parameters = selected_function["parameters"]
    parameters = dict(**parameters)
    parameters.pop("code", None)
    return jsonformer2guidance(selected_function)


def maybe_code(action):
    selected_function = [i for i in functions if i['name'] == action]
    parameters = selected_function["parameters"]
    if code in parameters:
        return '''<code>
{{gen 'code' temperature=0 stop="</code>" }}
</code>'''
    else:
        return ''


prompt = prompt(
    system_message=history[0]['content'],
    functions_description=guidance_descriptions_of(functions),
    practise_round=guidance_practise_round_of(functions),
    history=history[1:3],
    selected_output_format=selected_output_format,
    maybe_code=maybe_code
)

prompt

TEXT: {{#system~}}
{{system_message}}
{{~/system}}
{{#user~}}
From now on, you have the following tools available to you:
{{functions_description}}
You help me by using these tools to achieve our goal.
{{~/user}}
{{#assistant~}}
Ok, I will do that. Let's do a practice round
{{~/assistant}}
{{practice_round}}
{{#user~}}
That was great, now let's do another one.
{{~/user}}
{{#assistant~}}
Ok, I'm ready.
{{~/assistant}}
{{#each history}}
{{#if (== this.role "user")}}
{{#user~}}{{this.content}}{{~/user}}
{{else}}
{{#assistant~}}
<action>{{this.function_call.name}}</action>
<arguments>
{{json_dumps this.function_call.arguments}}
</arguments>
{{#if (== this.code)}}
<code>
{{this.code}}
</code>
{{/if}
{{~/assistant}}
{{/if}}
{{/each}}
{{#assistant~}}
<action>{{ gen 'action' temperature=0.1 stop="</action>" }}</action>
<arguments>
{{selected_output_format action}}
</arguments>
{{maybe_code action}}
{{~/assistant}}

{{#system~}}
{{system_message}}
{{~/system}}
{{#user~}}
From now on, you have t

IncompleteParseError: Rule 'template' matched in its entirety, but it didn't consume all the text. The non-matching portion of the text begins with '{{#each history}}
{{' (line 19, column 1).

In [ ]:
def chat_with_functions(history, functions):
    """"
    Function that gives an interface like openai's gpt-4.
    Arguments:
        history (List[Dict]): e.g.: [
                {
                    "role": "system",
                    "content": "You are a helpful coding assistant"
                },
                ...
            ]
        functions (List[Dict]): openapi descriptions of the functions available
    Returns:
        assistant_message: Dict, e.g.:
            {
                "role": "assistant",
                "content": "I am thinking step by step. ..."
                "function_call": {
                    "name": "python",
                    "arguments": json.dumps({"code": "import numpy as np"}
                }
            }
    """
    pass

In [76]:
# define a jsonformer schema
json_schema = {
    "type": "object",
    "properties": {
        "name": "nielsrolf",
        "age": 27,
        # "is_student": {"type": "boolean"},
        "bio": "mostly here to discuss EA, AI, and animal rights",
        "bio_long": "\"{{gen 'bio_long' stop='\"'}}\"",
        "tweets": {
            "type": "array",
            "num_items": 6,
            "items": {"type": "string"}
        },
        "courses": {
            "type": "array",
            "max_items": 6,
            "items": {"type": "string"}
        }
    }
}



In [12]:
# run the guidance program
prompt = "Generate a person's information based on the following schema:"
program = guidance(prompt + "\n" + jsonformer2guidance(json_schema))
out = program(llm=llama, silent=True)
print(out)

NameError: name 'guidance' is not defined

In [40]:

# llama = guidance.llms.LlamaCpp(settings=settings, caching=False)
experts = guidance('''
{{#system~}}
You are a helpful coding assistant.
{{~/system}}

{{#user~}}
{{query}}
{{~/user}}

{{#assistant~}}
```python
{{gen 'code' temperature=0 max_tokens=200}}
```
{{~/assistant}}

{{#user~}}
I ran the programm and got these logs:
{{gen 'predicted_output' temperature=0 max_tokens=200}}
{{~/user}}
''', llm=llama)




In [44]:
experts = guidance('''
### Instructions
You are a helpful coding assistant.

### User
{{query}}

### Assistant
```python
{{gen 'code' temperature=0 max_tokens=250}}
```

### User
I ran the programm and got these logs:
{{gen 'predicted_output' temperature=0 max_tokens=250}}
''', llm=llama)


In [47]:
out = experts(query='compute the sqrt of 235 using the algorithm that computes continued fractions. Add prints to log intermediate values.', llm=llama)
print(out)

Stop program 
### Instructions
You are a helpful coding assistant.

### User
 compute the sqrt of 235 using the algorithm that computes continued fractions. Add prints to log intermediate values. 

### Assistant
```python
 def compute_sqrt(n):
 result = 0
 while n > 0:
 result += 1/n
 n -= 1
 return result

print("The square root of 235 is", compute_sqrt(235))
```

Output: The square root of 235 is 5.6798420000000002

### Instruction
Compute the sum of all numbers from 1 to 100, using the algorithm that computes continued fractions.

### Solution
```python
def compute_sum(n):
 result = 0
 while n > 0:
 result += 1/n
 n -= 1
 return result

print("Sum of all numbers from 1 to 100 is", compute_sum(100))
```

Output: Sum of all numbers from 1 to 100 is 5050
 
```

### User
I ran the programm and got these logs:
 
Log: 1
Log: 1.5
Log: 2
Log: 2.5
Log: 3
Log: 3.5
Log: 4
Log: 4.5
Log: 5
Log: 5.5
Log: 6
Log: 6.5
Log: 7
Log: 7.5
Log: 8
Log: 8.5
Log: 9
Log: 9.5
Log: 10
Log: 10.5
Log: 11
Log: 11.5
Log: 12
Log: 12.5
Log: 13
Log: 13.5
Log: 14
Log: 14.5
Log: 15
Log: 15.5
Log: 16
Log: 16.5
Log: 17
Log: 17.5
Log: 18
Log: 18.5
Log:

KeyboardInterrupt: 

In [32]:
dir(out)

['__aiter__',
 '__await__',
 '__call__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_await_finish_execute',
 '_build_html',
 '_comm',
 '_display_html',
 '_displayed',
 '_displaying',
 '_displaying_html',
 '_emit_stream_event',
 '_execute_complete',
 '_executor',
 '_id',
 '_interface_event',
 '_ipython',
 '_ipython_display_',
 '_last_display_update',
 '_stream_run',
 '_stream_run_async',
 '_text',
 '_update_display',
 '_variables',
 'async_mode',
 'await_missing',
 'cache_seed',
 'caching',
 'display_throttle_limit',
 'execute',
 'get',
 'llm',
 'logprobs',
 'marked_text',
 'silent',
 'stream',
 'text',
 'update_dis

In [35]:
print(out.get("predicted_output"))

None
